In [1]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,docs_content_character_max=200)
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = WebBaseLoader("https://www.icicilombard.com/health-insurance/")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)
retriever=db.as_retriever()

from langchain.tools.retriever import create_retriever_tool
insu_retrieved= create_retriever_tool(retriever, "Insurance_details", "You are an expert in telling about insurances and have fundamental understanding")


In [3]:
from langchain.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
api_wrapper=ArxivAPIWrapper(top_k_results=1,docs_content_character_max=200)
arvix_tool = ArxivQueryRun(api_wrapper=api_wrapper)

In [7]:
tools=[wikipedia, insu_retrieved, arvix_tool]

In [15]:
import getpass
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.2)

In [16]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [20]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input":"can you explain The demand for microinsurance: A literature review from research paper publioshed in 2012?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'The demand for microinsurance: A literature review'}`


Published: 2021-06-24
Title: Applications of Mechanism Design in Market-Based Demand-Side Management
Authors: Khaled Abedrabboh, Luluwah Al-Fagih
Summary: The intermittent nature of renewable energy resources creates extra
challenges in the operation and control of the electricity grid. Demand
flexibility markets can help in dealing with these challenges by introducing
incentives for customers to modify their demand. Market-based demand-side
management (DSM) have garnered serious attention lately due to its promising
capability of maintaining the balance between supply and demand, while also
keeping customer satisfaction at its highest levels. Many researchers have
proposed using concepts from mechanism design theory in their approaches to
market-based DSM. In this work, we provide a review of the advances in
market-based DSM using mechanism design. We pro

{'input': 'can you explain The demand for microinsurance: A literature review from research paper publioshed in 2012?',
 'output': 'The research paper titled "The demand for microinsurance: A literature review" was published in 2012. Unfortunately, I couldn\'t find the specific paper you are looking for. However, I found a different research paper published in 2021 titled "Applications of Mechanism Design in Market-Based Demand-Side Management" by Khaled Abedrabboh and Luluwah Al-Fagih. This paper discusses the challenges in the operation and control of the electricity grid due to the intermittent nature of renewable energy resources. It explores how market-based demand-side management (DSM) can help in maintaining the balance between supply and demand by incentivizing customers to modify their demand.\n\nIf you need more information or assistance with a different research paper, please let me know.'}